In [1]:
import pandas_datareader as pdr
from pandas_datareader import data, wb
from datetime import date
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
stocks = ['FB','KO','TSM','SFTBY','INTC','SNE','ET','CL','RYCEY','SIEGY','GE','FUJIY']

In [20]:
def calc_beta(stock,benchmark):
    stock_data = pdr.get_data_yahoo(stock,'2015-01-01','2019-01-01')
    benchmark_data = pdr.get_data_yahoo(benchmark,'2015-01-01','2019-01-01')
    stock_data = stock_data.resample('M').last()
    benchmark_data = benchmark_data.resample('M').last()

    data = pd.DataFrame({'stock close':stock_data['Adj Close'],'benchmark close':benchmark_data['Adj Close']}, index = stock_data.index)

    data['y'] = data['stock close']/data['stock close'].shift(1)
    data['X'] = data['benchmark close']/data['benchmark close'].shift(1)

    data = data.dropna()

    beta = np.cov(data['X'],data['y'])[0,1]/np.var(data['X'])
    #beta, alpha = np.polyfit(data['X'],data['y'],deg=1)

    return beta
    

In [23]:
betas = []
for stock in stocks:
    betas.append(calc_beta(stock,'^GSPC'))
stock_beta = {stocks[i]:betas[i] for i in range(12)}
beta_rank = sorted(stock_beta.items(), key=lambda item: item[1])
print(beta_rank)

[('FUJIY', 0.3977890483546764), ('KO', 0.5259249746425118), ('GE', 0.5901880030106319), ('FB', 0.730276385875083), ('CL', 0.7366471722574207), ('INTC', 0.8023816896605757), ('TSM', 1.0312837778634172), ('RYCEY', 1.1042877806135607), ('SIEGY', 1.1947367088991536), ('SFTBY', 1.602913003453671), ('SNE', 1.7091685475315497), ('ET', 1.8548517956605863)]


In [24]:
bot = [x[0] for x in beta_rank[:3]]
top = [x[0] for x in beta_rank[-3:]]

In [26]:
def eval(stocks,benchmark):
    returns = []
    for stock in stocks:
        stock_data = pdr.get_data_yahoo(stock,'2019-01-01','2021-01-01')
        stock_return = np.log(stock_data['Adj Close'][-1]/stock_data['Adj Close'][0])
        returns.append((stock,stock_return))
    return returns

benchmark_data = pdr.get_data_yahoo('^GSPC','2019-01-01','2021-01-01')
benchmark_return = np.log(benchmark_data['Adj Close'][-1]/benchmark_data['Adj Close'][0])


In [27]:
top_returns = eval(top,'^GSPC')
bot_returns = eval(bot,'^GSPC')
print('High beta returns:', top_returns)
print('Low beta returns:', bot_returns)
print('Benchmark returns:', benchmark_return)

High beta returns: [('SFTBY', 0.8569145740297363), ('SNE', 0.7300204624765328), ('ET', -0.542176868271763)]
Low beta returns: [('FUJIY', 0.29965140898454795), ('KO', 0.22060543473819183), ('GE', 0.3421639401933908)]
Benchmark returns: 0.4030784992843473
